In [1]:
import re
import json
from parse_bitstream_v2 import BitstreamParser
import hashlib
from collections import defaultdict

In [31]:
import config_command

In [32]:

config_command.asf()

AttributeError: module 'config_command' has no attribute 'asf'

In [2]:
json_file_path = r"E:\workspace\parse_bitstream\parse_bitstream\压缩位流相关知识\MCF100T_v2.json"
# 读取JSON文件
with open(json_file_path, 'r') as file:
    json_file = json.load(file)

In [3]:
file_path = r"E:\workspace\parse_bitstream\parse_bitstream\压缩位流相关知识\compress\原版\origin.rbt"
bit_parser = BitstreamParser(file_path, "False")

In [4]:
data_frame = bit_parser.rbt_data_content

In [5]:
# 获取特征值
# 这里传入content为list， 每个元素的数据类型为 content_type
def get_feature(content:list, content_type = "int"):
    if content_type == "str":
        # 此时是二进制字符串作为一个元素
        combined_string = ''.join(content)
        # 使用 hashlib 计算特征值（SHA256）
        feature_hash = hashlib.sha256(combined_string.encode()).hexdigest()
        return feature_hash
    elif content_type == "int":
        pass

In [6]:
# 数据帧特征值作为key
# value : 
# {   
#     "FAR": {
#         "frame_type": 0,
#         "region_type": 0,
#         "row_num": 0,
#         "col_num": 0,
#         "frame_index": 0
#     },
#     "index":{
#         "start_index":0,
#         "end_index":101
#     },
#     "feature": "123"
# }
# 每个row相互独立
data_frame_features_index = defaultdict(list)
start_index = 0
end_index = 101 # 这里是开区间，不包括101
cur_row = 0
frame_count = 0
all_frame_count = 0
for frame_type_key, frame_type_value in json_file.items():
    
    # type 0 type 1
    data_frame_features_index[frame_type_key] = {}
    
    # 遍历 frame_type 层
    for region_type_key, region_type_value in frame_type_value.items():
        
        # top bottom
        data_frame_features_index[frame_type_key][region_type_key] = {}
        
        # 遍历 region_type 层
        for row_num_key, row_num_value in region_type_value.items():
            
            # row 0 1
            data_frame_features_index[frame_type_key][region_type_key][row_num_key] = defaultdict(list)
            frame_count = 0
            # 遍历 row_num 层
            for col_num_key, group_data_item in row_num_value.items():
                # group_data_item 包含 FAR 和 frame_count
                frame_count += group_data_item["frame_count"]
                word_count = group_data_item["frame_count"] * 101 # 这里拿到word数
                end_index = start_index + word_count
                
                # 每次拿一帧的数据计算
                frame_index = 0
                for index in range(start_index, end_index, 101):
                    feature = get_feature(data_frame[index:index+101], "str")
                    
                    # 添加到字典
                    data_frame_features_index[frame_type_key][region_type_key][row_num_key][feature].append(
                        {
                            "FAR": {
                                "frame_type": group_data_item["FAR"]["frame_type"],
                                "region_type": group_data_item["FAR"]["region_type"],
                                "row_num": group_data_item["FAR"]["row_num"],
                                "col_num": group_data_item["FAR"]["col_num"],
                                "frame_index": frame_index
                            },
                            "index": {
                                "start_index": index,
                                "end_index": index + 101
                            },
                            "feature": feature
                        }
                    )
                    frame_index += 1
                
                # 更新 start_index
                start_index = end_index
                
            # 完成一个row后，start_index需要向后跳两帧，这里有两帧的pad
            start_index += 101*2
            
            all_frame_count += frame_count

In [7]:
new_data_frame = []

In [16]:
# 插入数据，方便维护
def insert_data(target,content,content_type="int"):
    if content_type == "str":
        target.append(content)
    elif content_type == "int":
        pass
    else:
        pass

['asf']


In [2]:

def create_frame_address_register_binary(type, tb, row, col, min_value):
    # 构造32位二进制数
    binary = (type << 23) | (tb << 22) | (row << 17) | (col << 7) | min_value
    # 转换为32位二进制字符串
    return format(binary, '032b')

create_frame_address_register_binary(0,0,0,0,1)

'00000000000000000000000000000001'

In [15]:

for frame_type_key, frame_type_value in data_frame_features_index.items():
    # 遍历 frame_type 层
    for region_type_key, region_type_value in frame_type_value.items():
        # 遍历 region_type 层
        for row_num_key, row_num_value in region_type_value.items():
            # 遍历 row_num 层,每个元素
            # feature : 
            # [{   
            #     "FAR": {
            #         "frame_type": 0,
            #         "region_type": 0,
            #         "row_num": 0,
            #         "col_num": 0,
            #         "frame_index": 0
            #     },
            #     "index":{
            #         "start_index":0,
            #         "end_index":101
            #     },
            #     "feature": "123"
            # },
            # ...]
            for feature,group_data_item in row_num_value.items():
                # group_data_item 为 每个row不同特征值的数据
                # 当前特征值下帧数目
                group_len = len(group_data_item)
                if group_len > 1:
                    # 多帧数据
                    # 头帧格式：
                    # 00110000000000000010000000000001	Write Reg:FAR, word:1
                    # 00000000000000000000000000000000	type:0 tb:0 row:0 col:0 min:0 真实地址，来自于FAR
                    # 00110000000000001000000000000001	Write Reg:CMD, word:1
                    # 00000000000000000000000000000001	command:WCFG
                    # 00100000000000000000000000000000	NOP*1
                    # 00110000000000000100000001100101	Write Reg:FDRI, word:101
                    # ........... 101 WORD ...........
                    # 00110000000000001000000000000001	Write Reg:CMD, word:1
                    # 00000000000000000000000000000010	command:MFW 
                    # 00100000000000000000000000000000	NOP*12
                    # 00110000000000010100000000001000	Write Reg:MFWR, word:8
                    # 00000000000000000000000000000000  ZERO_FRAME*8
                    
                    # 1. 插入一个完整头帧
                    
                    # 
                    pass
                elif group_len == 1:
                    # 单帧时
                    
                    pass
                else:
                    # 异常情况
                    
                    pass

SyntaxError: invalid syntax (782108347.py, line 31)

In [37]:
# for item in data_frame_features_index["frame_type_0"]["region_type_0"]["row_num_0"]["077a6de9ffaa89c2626a4dc887d001fd81cf8c41a9bf796acfba47ed385b4f79"]:
#     print(item['FAR'])
#     print(data_frame[item['index']['start_index']:item['index']['end_index']])

In [10]:
def get_frame_data_register_input(word_count):
    # 将 word_count 转换为 10 位的二进制字符串，不足 10 位补 0，超出则截取
    word_count_binary = f'{word_count:011b}'[-11:]
    # 固定部分：前 22 位是 "001100000000000001000"
    fixed_part = "001100000000000001000"
    
    # 拼接前 22 位和后 10 位的 word_count
    frame_data_register_input = fixed_part + word_count_binary
    
        # 将二进制字符串转换为整数
    frame_data_register_input_int = int(frame_data_register_input, 2)

    # 将整数转换为 4 字节的字节序列
    frame_data_register_input_bytes = frame_data_register_input_int.to_bytes(4, byteorder='big')

    return frame_data_register_input_bytes

get_frame_data_register_input(90)

b'0\x00@Z'

In [9]:
f'{99999999999999999999999:011b}'[-11:]

'11111111111'

In [ ]:
a = {"a":123,"b":123,"c":123,"d":123}